In [1]:
import pandas as pd
import os
import folium
from folium import plugins

In [4]:
PATH = os.getcwd()
df = pd.read_csv(f"{PATH}/data/metro.csv", sep=",")
df.head()

,Station,Line,Order of Points,Longitude,Latitude,Traffic
0,Pinar de Chamartin,Linea 1,1,"-3,66706111","40,48013631",6.222.074
1,Bambú,Linea 1,2,"-3,676373739","40,47687195",1.220.848
2,Chamartín,Linea 1,3,"-3,682768126","40,47213836",8.797.197
3,Plaza de Castilla,Linea 1,4,"-3,689162512","40,4668494",25.135.236
4,Valdeacederas,Linea 1,5,"-3,695127745","40,46440066",3.877.235


In [5]:
# Primero arreglamos el formato de los datos
# Las columnas de coordenadas tienen comas en lugar de puntos decimales
df['Longitude'] = df['Longitude'].str.replace(',', '.').astype(float)
df['Latitude'] = df['Latitude'].str.replace(',', '.').astype(float)
# La columna de tráfico tiene puntos que separan miles
df['Traffic'] = df['Traffic'].str.replace('.', '').astype(int)

df.head()

,Station,Line,Order of Points,Longitude,Latitude,Traffic
0,Pinar de Chamartin,Linea 1,1,-3.667061,40.480136,6222074
1,Bambú,Linea 1,2,-3.676374,40.476872,1220848
2,Chamartín,Linea 1,3,-3.682768,40.472138,8797197
3,Plaza de Castilla,Linea 1,4,-3.689163,40.466849,25135236
4,Valdeacederas,Linea 1,5,-3.695128,40.464401,3877235


In [10]:
# Función auxiliar para ordenar las líneas correctamente
def ordenar_lineas(linea):
    return int(linea.replace('Linea ', ''))

In [12]:
# Creamos el mapa con las conexiones
mapa_metro = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=12
)

# Colores oficiales del Metro de Madrid
colores_linea = {
    'Linea 1': '#2B7CE9',    # Azul claro
    'Linea 2': '#E6343C',    # Rojo
    'Linea 3': '#FFD700',    # Amarillo
    'Linea 4': '#8B4513',    # Marrón
    'Linea 5': '#4CAF50',    # Verde
    'Linea 6': '#808080',    # Gris
    'Linea 7': '#FF8C00',    # Naranja
    'Linea 8': '#FFC0CB',    # Rosa
    'Linea 9': '#800080',    # Morado
    'Linea 10': '#000080',   # Azul oscuro
    'Linea 11': '#90EE90',   # Verde claro
    'Linea 12': '#DAA520'    # Dorado más oscuro (GoldenRod)
}

# Creamos las capas para cada línea
for linea in sorted(df['Line'].unique(), key=ordenar_lineas):  # Ordenamos usando la función auxiliar
    
    # Creamos un grupo para esta línea
    line_group = folium.FeatureGroup(name=f'{linea}')
    
    # Filtramos y ordenamos las estaciones de esta línea
    df_linea = df[df['Line'] == linea].sort_values('Order of Points')
    
    # Primero dibujamos las conexiones entre estaciones
    coordinates = df_linea[['Latitude', 'Longitude']].values.tolist()
    if len(coordinates) > 1:
        folium.PolyLine(
            locations=coordinates,
            color=colores_linea[linea],
            weight=3,
            opacity=0.8
        ).add_to(line_group)
    
    # Calculamos el rango de tráfico para esta línea
    min_traffic = df_linea['Traffic'].min()
    max_traffic = df_linea['Traffic'].max()
    min_radius = 5
    max_radius = 30
    
    # Luego añadimos las estaciones
    for _, row in df_linea.iterrows():
        # Calculamos el radio proporcional al tráfico
        radius = min_radius + (row['Traffic'] - min_traffic) * (max_radius - min_radius) / (max_traffic - min_traffic)
        
        # Añadimos el marcador de la estación
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=radius,
            popup=f"Estación: {row['Station']}<br>Línea: {row['Line']}<br>Tráfico: {row['Traffic']:,} pasajeros",
            color=colores_linea[linea],
            fill=True,
            fill_color=colores_linea[linea],
            fill_opacity=0.2,
            weight=3,
            tooltip=f"{row['Station']}: {row['Traffic']:,} pasajeros"
        ).add_to(line_group)
    
    line_group.add_to(mapa_metro)

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_metro)

# Mostramos el mapa
display(mapa_metro)

Número de estaciones por línea:
Line
Linea 1     33
Linea 10    31
Linea 11     7
Linea 12    29
Linea 2     20
Linea 3     18
Linea 4     23
Linea 5     32
Linea 6     29
Linea 7     31
Linea 8      8
Linea 9     24
dtype: int64


In [21]:
# Creamos el mapa con las conexiones
mapa_metro = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=12
)

# Colores oficiales del Metro de Madrid
colores_linea = {
    'Linea 1': '#2B7CE9',    # Azul claro
    'Linea 2': '#E6343C',    # Rojo
    'Linea 3': '#FFD700',    # Amarillo
    'Linea 4': '#8B4513',    # Marrón
    'Linea 5': '#4CAF50',    # Verde
    'Linea 6': '#808080',    # Gris
    'Linea 7': '#FF8C00',    # Naranja
    'Linea 8': '#FFC0CB',    # Rosa
    'Linea 9': '#800080',    # Morado
    'Linea 10': '#000080',   # Azul oscuro
    'Linea 11': '#90EE90',   # Verde claro
    'Linea 12': '#DAA520'    # Dorado más oscuro (GoldenRod)
}

# Creamos las capas para cada línea
for linea in sorted(df['Line'].unique(), key=ordenar_lineas):  # Ordenamos usando la función auxiliar
    
    # Creamos un grupo para esta línea
    line_group = folium.FeatureGroup(name=f'{linea}')
    
    # Filtramos y ordenamos las estaciones de esta línea
    df_linea = df[df['Line'] == linea].sort_values('Order of Points')
    
    # Primero dibujamos las conexiones entre estaciones
    coordinates = df_linea[['Latitude', 'Longitude']].values.tolist()
    if len(coordinates) > 1:
        folium.PolyLine(
            locations=coordinates,
            color=colores_linea[linea],
            weight=3,
            opacity=0.8
        ).add_to(line_group)
    
    # Calculamos el rango de tráfico para esta línea
    min_traffic = df_linea['Traffic'].min()
    max_traffic = df_linea['Traffic'].max()
    min_radius = 5
    max_radius = 30
    
    # Luego añadimos las estaciones
    for _, row in df_linea.iterrows():
        # Calculamos el radio proporcional al tráfico
        radius = min_radius + (row['Traffic'] - min_traffic) * (max_radius - min_radius) / (max_traffic - min_traffic)
        
        # Añadimos el marcador de la estación
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=radius,
            popup=f"Estación: {row['Station']}<br>Línea: {row['Line']}<br>Tráfico: {row['Traffic']:,} pasajeros",
            color=colores_linea[linea],
            fill=True,
            fill_color=colores_linea[linea],
            fill_opacity=0.2,
            weight=3,
            tooltip=f"{row['Station']}: {row['Traffic']:,} pasajeros"
        ).add_to(line_group)
    
    line_group.add_to(mapa_metro)

# Añadimos el control de capas
folium.LayerControl(collapsed=False).add_to(mapa_metro)

# Calculamos estadísticas por línea
stats_por_linea = df.groupby('Line').agg({
    'Traffic': ['sum', 'mean', 'max', 'min', 'count']
}).round(0)

# Formateamos los números para mejor legibilidad
def format_number(num):
    if num >= 1_000_000:
        return f"{num/1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num/1_000:.0f}K"
    return str(int(num))

# Creamos el HTML para el panel de estadísticas
stats_html = '''
<div style="position: fixed; 
            bottom: 50px; 
            left: 50px; 
            width: 300px;
            max-height: 500px;
            overflow-y: auto;
            z-index: 1000;
            background-color: white;
            padding: 10px;
            border-radius: 5px;
            border: 2px solid grey;
            font-size: 12px;">
    <p style="margin-bottom: 10px;"><strong>Estadísticas por Línea</strong></p>
    <table style="width: 100%; border-collapse: collapse;">
        <tr style="background-color: #f2f2f2;">
            <th style="padding: 5px; text-align: left; border-bottom: 1px solid #ddd;">Línea</th>
            <th style="padding: 5px; text-align: right; border-bottom: 1px solid #ddd;">Total</th>
            <th style="padding: 5px; text-align: right; border-bottom: 1px solid #ddd;">Promedio</th>
            <th style="padding: 5px; text-align: right; border-bottom: 1px solid #ddd;">Estaciones</th>
        </tr>
'''

# Añadimos cada línea a la tabla
for linea in sorted(df['Line'].unique(), key=lambda x: int(x.replace('Linea ', ''))):
    stats = stats_por_linea.loc[linea]
    stats_html += f'''
        <tr style="border-bottom: 1px solid #ddd;">
            <td style="padding: 5px;">
                <span style="color: {colores_linea[linea]}">●</span> {linea}
            </td>
            <td style="padding: 5px; text-align: right;">{format_number(stats['Traffic']['sum'])}</td>
            <td style="padding: 5px; text-align: right;">{format_number(stats['Traffic']['mean'])}</td>
            <td style="padding: 5px; text-align: right;">{int(stats['Traffic']['count'])}</td>
        </tr>
    '''

stats_html += '''
    </table>
</div>
'''

# Añadimos el panel de estadísticas al mapa
mapa_metro.get_root().html.add_child(folium.Element(stats_html))


# Mostramos el mapa
display(mapa_metro)